In [125]:
## parameters for experiment
N_BLOCK = 5
LR = 0.001

OUTPUT_DIR = '20251204-CorrectionNetworkTestv2.3'

RANDOM_SEED = 42

IN_CHANNEL = 1

AUGMENTED = False
AUGMENTATION  =  30

CROSS_VAL = False
N_SPLIT = 4

MINE_EPOCH = 25 # start mining after this epoch
MAX_MINE_TRAIN = 22
MAX_MINE_VAL = 11
MIN_ERROR_THRESHOLD = 150  

In [126]:
## import libraries
from numpy.core.numeric import NaN
from MCtool.RFilter import gray
from genericpath import exists
from matplotlib import image
import math
import sys
import time

import cv2
from matplotlib import pyplot as plt
from tensorflow.python.keras.backend import dtype
from DeepLearning import LearnAndTest
from Rpkg.Rfund.InputFeature import InputFeature
import datetime
import os
import gc
import tensorflow as tf
import random
import numpy as np
import pandas as pd

from Rpkg.Rfund import ReadFile, WriteFile
from Rpkg.Rmodel import Unet, Mnet

import Filtering

import torch
from torch import nn


import DeepLearning
from tensorflow.keras.optimizers import Adam

from Rpkg.Rfund.InputFeature import InputFeature
from Rpkg.Rfund import ReadFile, WriteFile
from Rpkg.Rmodel import Unet, Mnet

from MCtool import RFilter, resultEval
from DeepLearning import save_eval_result

import numpy as np
import cv2
import torch
from transformations import ComposeDouble, FunctionWrapperDouble, create_dense_target, normalize_01
from customdatasets import SegmentationDataSet1
from customdatasets import SegmentationDataSet4
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pathlib
from skimage.transform import resize

#early stopping なし
from unet import UNet
from trainer import Trainer
from sklearn.model_selection import StratifiedKFold, train_test_split




In [127]:
## random seed config
# Make sure there is no randomness in the output so that the output is reproduceable
import torch
import numpy as np
import random

# Set seed for Python random module
random.seed(RANDOM_SEED)

# Set seed for NumPy
np.random.seed(RANDOM_SEED)

# Set seed for PyTorch
torch.manual_seed(RANDOM_SEED)

# If you are using GPU
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

# Make the convolution operations deterministic
torch.backends.cudnn.deterministic = True

# Disable the CUDNN benchmark to ensure deterministic results
torch.backends.cudnn.benchmark = False


In [128]:
## cuda and pytorch stats
# 自分の環境設定がうまくいったかどうかを確認しましょう、特にGPUの動作
# Prints the version of PyTorch installed
print('PyTorch Version installed: ' + torch.__version__)

# Prints the version of CUDA associated with the installed PyTorch version
print('CUDA version associated with PyTorch version: ' + torch.version.cuda)

# Prints the version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch
print('Version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch' + str(torch.backends.cudnn.version()))

# Same as the line above
print('CUDA is available: ' + str(torch.cuda.is_available()))

# Returns the number of available CUDA-enabled GPUs
print('Number of GPUs compatible with CUDA:' + str(torch.cuda.device_count()))

# Returns the name of the GPU at index 0
print('Name of the GPU at index 0: '  + str(torch.cuda.get_device_name(0)))

# Returns the index of the current CUDA device being used
print('Current CUDA device index: '  + str(torch.cuda.current_device()))


PyTorch Version installed: 2.3.0+cu121
CUDA version associated with PyTorch version: 12.1
Version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch8902
CUDA is available: True
Number of GPUs compatible with CUDA:1
Name of the GPU at index 0: NVIDIA GeForce RTX 2080 Ti
Current CUDA device index: 0


In [129]:
## file_names_with_prefix
# ファイル名の先頭部分（prefix）により自動的にファイル名を抽出するアルゴリズム。
# 実際それぞれのファイル名は違うと思うので、必須ではない

""" 

Extracts filenames in directory if they start with the prefix input 


Args/Parameters:

    directory_path (string): The path of the dir (ex: /root/home/Documents/etc)
    
    prefix (string): Prefix of the file name (ex: 'Bo' is a prefix of 'Bone')

Returns:

    sorted_file_names (list of str): File names sorted in ascending order in the dir without extension ex: ['bone1', 'bone2', ...]

Raises:

    SomeError: ...

"""

def file_names_with_prefix(directory_path, prefix):

    # Initialize an empty list to store the file names without extensions
    file_names_without_extension = []

    # Loop through all files in the specified directory
    for filename in os.listdir(directory_path):
        #Checking if the file in loop exists in the directory_path not sure how is this necessary
        #??
        if os.path.isfile(os.path.join(directory_path, filename)):
            # Check if the file name starts with the specified prefix
            if filename.startswith(prefix):
                # Get the file name without extension
                name_without_extension, _ = os.path.splitext(filename)

                # Append the file name (without extension) to the list
                file_names_without_extension.append(name_without_extension)

    # Sort the list of file names without extensions in ascending order
    sorted_file_names = sorted(
        file_names_without_extension,
        key=lambda x: (x.split('-')[0], int(x.split('-')[1]))
    )  # Modify this part based on your file naming convention

    # Now you have a sorted list of file names with the specified prefix and without extensions
    return sorted_file_names


In [130]:
## paths config
import pathlib
from pathlib import Path

# ここで、folder名とかPathとか色々設定

# Setting the directory name, path and other settings

# Define the root directory where your project is located
# Defining a Path object for the project's root dir
root_dir = Path(pathlib.Path.cwd())

# result folder name
#date_str = '20241202-Conv1x1-' + str(OUTPUT_DIR)
date_str = OUTPUT_DIR

# Define the directories for different types of data
# Concatenating the root dir to the different dataset dirs
data_dir = str(root_dir / "img_1006t/original")
feature_dir = str(root_dir / "img_1006t/feature") 
labeled_dir = str(root_dir / "img_1006t/labeled")

augmented_labeled_dir = str(root_dir / "img_1006t/labelAug")
augmented_data_dir = str(root_dir / "img_1006t/originalAug")
augmented_feature_dir = str(root_dir / "img_1006t/featureAug")


# data_dir = str(root_dir / "img_1006t/MC3x3(108_3out)/mcOriginal")
# feature_dir = str(root_dir / "img_1006t/MC3x3(108_3out)/mcFeature") 
# labeled_dir = str(root_dir / "img_1006t/labeled")

# augmented_labeled_dir = str(root_dir / "img_1006t/MC3x3(108_3out)/augMCLabel")
# augmented_data_dir = str(root_dir / "img_1006t/MC3x3(108_3out)/augMCOriginal")
# augmented_feature_dir = str(root_dir / "img_1006t/MC3x3(108_3out)/augMCFeature")

# annealing_img_dir = str(root_dir / "img_1006/annealing_img") # 焼きなまし法時に使う
# annealing later, original for now
annealing_img_dir = str(root_dir / "img_1006/original")
result_dir = str(root_dir / "result" / date_str)
test_result_dir= str(root_dir / "result_test" / date_str)

# Making directories based on the path string result_dir and test_result_dir
Path(result_dir).mkdir(parents=True, exist_ok=True)
Path(test_result_dir).mkdir(parents=True, exist_ok=True)

# Prints the paths of the dirs
print('Root directory: ' + str(root_dir))
print('Data directory (original dir): ' + str(data_dir))
print('Feature img directory: ' + str(feature_dir))
print('Labeled img directory: ' + str(labeled_dir))
print('Annealing directory: ' + str(annealing_img_dir))
print('Result directory: ' + str(result_dir))
print('Test result directory: ' + str(test_result_dir))

# Defining variables filename list of path str starts with the prefix format
# In this case: N1 and N3 is training data and N2 is validation data and N4 is a test data
input_train = []
input_name_val = []
annealing_input_name = []
input_train = []
test_input_name = []


# for raw_input_img in INPUT:
#     input_train.extend(file_names_with_prefix(data_dir, raw_input_img))
# for raw_val_img in VALIDATION:
#     input_name_val.extend(file_names_with_prefix(data_dir, raw_val_img))
# for raw_anneal in ANNEALING:
#     annealing_input_name.extend(file_names_with_prefix(data_dir, raw_anneal))
# for raw_test in TEST:
#     test_input_name.extend(file_names_with_prefix(data_dir, raw_test))


####old version of assigning
# input_train = file_names_with_prefix(data_dir, INPUT)
# input_name_val = file_names_with_prefix(data_dir, VALIDATION)
# annealing_input_name = file_names_with_prefix(data_dir, ANNEALING)
# test_input_name = file_names_with_prefix(data_dir, TEST) 


# extra_dataset = file_names_with_prefix(data_dir,'N5-')
# input_train.extend(extra_dataset)

# Prints the each data image name
# print(input_train)
# print(input_name_val)
# print(annealing_input_name)
# print(test_input_name)
# print(extra_dataset)


# Defining a var to store each list length
len_train = len(input_train)
len_val = len(input_name_val)
len_test = len(test_input_name)
len_annealing = len(annealing_input_name)


# print(len(input_train))

# print(len(input_name_val))
# print(len(test_input_name))
# print(len(annealing_input_name))


Root directory: /home/eric/Documents/cervicalResearchIIP
Data directory (original dir): /home/eric/Documents/cervicalResearchIIP/img_1006t/original
Feature img directory: /home/eric/Documents/cervicalResearchIIP/img_1006t/feature
Labeled img directory: /home/eric/Documents/cervicalResearchIIP/img_1006t/labeled
Annealing directory: /home/eric/Documents/cervicalResearchIIP/img_1006/original
Result directory: /home/eric/Documents/cervicalResearchIIP/result/20251204-CorrectionNetworkTestv2.3
Test result directory: /home/eric/Documents/cervicalResearchIIP/result_test/20251204-CorrectionNetworkTestv2.3


In [131]:
## filters
# 特徴画像の特徴一覧をリストとして取得#
#inputfeature_list = list(map(str, InputFeature))
inputfeature_list = ['CLA_']

print(inputfeature_list)
feature_num = len(inputfeature_list)


print(feature_num)

['CLA_']
1


In [132]:
## createweightimage read by images
# takes too much memory cuz it loads all images into np array at once


## 重み計算なし
def CreateWeightImage(input_number, augmentation=False):
    print("Creating image arrays...")
    label_dataset = []
    arrDataset = []
    for i in input_number:
        if augmentation:
            label_path = os.path.join(augmented_labeled_dir, str(AUGMENTATION) + "aug/" , f"{i}.png")
        else:
            label_path = os.path.join(labeled_dir, f"{i}.png")
        input_originallabel = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
                    
        label_dataset.append(input_originallabel)

    print("Number of label images:", len(label_dataset))


    
    for i in input_number:
        # changed this part from 100 to 256

        dataset_img =  np.zeros((256, 256, feature_num), dtype=np.float32)
            
            
        for m in range(feature_num):
            if augmentation:
                feature_img_path = os.path.join(augmented_feature_dir, str(AUGMENTATION) + "aug/" , str(i), f"{inputfeature_list[m]}.png")
            else:
                feature_img_path = os.path.join(feature_dir, str(i), f"{inputfeature_list[m]}.png")
            input_featureimg = cv2.imread(feature_img_path, cv2.IMREAD_GRAYSCALE)
            dataset_img[:, :, m] = input_featureimg

        arrDataset.append(dataset_img)

    arrDataset = np.array(arrDataset)
    print("Completed creating image arrays:")
    print("Dataset shape ", arrDataset.shape)
    print("Label image shape ", np.shape(label_dataset))
    print()

    return arrDataset, label_dataset


In [133]:
## createweightimagenew read paths only
def CreateWeightImageNew(input_numbers, augmentation=False):
    print("Creating image paths...")
    label_paths = []
    feature_paths = []

    for i in input_numbers:
        if augmentation:
            label_path = os.path.join(augmented_labeled_dir, str(AUGMENTATION) + "aug/", f"{i}.png")
        else:
            label_path = os.path.join(labeled_dir, f"{i}.png")
        label_paths.append(label_path)

        feature_img_paths = []
        for feature_name in inputfeature_list:
            if augmentation:
                feature_img_path = os.path.join(augmented_feature_dir, str(AUGMENTATION) + "aug/", str(i), f"{feature_name}.png")
            else:
                feature_img_path = os.path.join(feature_dir, str(i), f"{feature_name}.png")
            feature_img_paths.append(feature_img_path)

        feature_paths.append(feature_img_paths)

    print(f"Processed {len(label_paths)} label paths and {len(feature_paths)} feature paths.")
    return feature_paths, label_paths


In [134]:
## minor irrelevant function
def print_model_shapes(model, input_tensor):
    def forward_hook(module, input, output):
        print(f"Layer: {module.__class__.__name__}")
        print(f"Input shape: {str(input[0].shape)}")
        print(f"Output shape: {str(output.shape)}")
        print("-----------------------")

    hooks = []
    for layer in model.children():
        hook = layer.register_forward_hook(forward_hook)
        hooks.append(hook)

    print("Model Architecture:")
    print(model)

    # Pass a dummy input tensor through the model to trigger the forward hooks
    with torch.no_grad():
        model(input_tensor)

    for hook in hooks:
        hook.remove()


In [135]:
## preprocess and postprocess function
def preprocess(img: np.ndarray):
    img = np.moveaxis(img, -1, 0)  # Change from [H, W, C] to [C, H, W]
    img = normalize_01(img)  # Linear scaling to range [0-1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension [B, C, H, W]
    img = img.astype(np.float32)  # Typecasting to float32
    #print("in pre")
    #print(np.unique(img))
    return img

# postprocess function
def postprocess(img: torch.tensor):
    img = torch.argmax(img, dim = 1)  # Perform argmax to generate 1 channel
    #img = img * 255.0 commented as the labels are from 0 to 11 in my case
    img = img.cpu().numpy().astype(np.uint8)  # Send to CPU and transform to numpy.ndarray
    # If batch_size > 1, you may need to loop through each batch and save them separately
    # If batch_size == 1, you can remove the batch dimension to save a single image

    # used for checking the unique label values whether if it is 0 to 11 or 0 to 255 scale
    #print("in post")
    #print(np.unique(img))


    img = np.squeeze(img)  # Remove batch dim and channel dim -> [H, W]
    # img = re_normalize(img)  # Scale it to the range [0-255]

    # If your image has multiple channels (C>1), like an RGB image, before saving with cv2.imwrite
    # you need to ensure the channel order is [B, G, R] instead of the common [R, G, B]
    # If C == 1, you can further reduce dimensions -> [H, W]
    if img.shape[0] == 3:  # [C, H, W]
        img = np.transpose(img, (1, 2, 0))  # [H, W, C]
        img = img[:, :, ::-1]  # Convert RGB to BGR
    elif img.shape[0] == 1:  # [C, H, W]
        img = np.squeeze(img, 0)  # [H, W]
    return img


In [136]:
def record_results(training_losses, validation_losses, lr_rates, learning_curve, model, fold, input_dataset_val, input_name_val, type_number, device, model_n = "unet1"):
    
    #logging to csv
    assert len(training_losses) == len(validation_losses) == len(lr_rates), \
        "Lengths of training, validation losses, and learning rates must match."

    # Create dataframe
    df = pd.DataFrame({
        "Epoch": list(range(1, len(training_losses) + 1)),
        "Training_Loss": training_losses,
        "Validation_Loss": validation_losses,
        "Learning_Rate": lr_rates
    })

    # Save to CSV
    df.to_csv(test_result_dir + f"/{model_n}_fold{fold}_log.csv", index = False)
    
    learning_curve.savefig(test_result_dir + f"/{model_n}_fold{fold}_learningcurve.png")
    
    print("***************************")

    
    
    
    #ここがちゃんとESで最適なエポック数のモデルになっているか要検証
    model_dir = os.path.join("model", date_str)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_name = f"{model_n}_fold_{fold}.pt"
    model_path = os.path.join(model_dir, model_name)
    torch.save(model.state_dict(), model_path)
    print(f"modelname:{model_name}を保存しました")
    #torch.cuda.empty_cache()
    

    model_weights = torch.load(model_path)
    model.load_state_dict(model_weights)
    
    

    # images = annealing_input_dataset
    images = input_dataset_val

    from inference import predict
    from transformations import normalize_01, re_normalize
    # predict the segmentation maps 
    # output = [predict(img, model, preprocess, postprocess, device) for img in images]


    # for i in range(len(input_name_val)):
    #     if(type_number == 0):
    #         cv2.imwrite(os.path.join(result_dir, f'{model_n}_{input_name_val[i]}.png'), output[i])
    #     elif(type_number == 1):
    #         cv2.imwrite(os.path.join(test_result_dir, 'result_original', f'{input_name_val[i]}.png'), output[i])
    #     elif(type_number == 2):
    #         cv2.imwrite(os.path.join(test_result_dir, 'result_test', f'{input_name_val[i]}.png'), output[i])

In [137]:
## learn_ea function training logic
# earlystoppingあり
# numpy形式のまま入力する用改良
# 学習を行い予測結果画像を出力するとこまで
from customdatasets import SegmentationDataSet0
from customdatasets import SegmentationDataSet1
from customdatasets import SegmentationDataSet5
from torch.utils.data import DataLoader

def Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val, type_number, fold=0, next_model = False, model_n = "unet1", inChannels = 1):
    print("*************************Training*************************")
    # 引数を追加して保存先を指定するよう改良
    # try_number:何回目の焼きなましかどうか。モデルの保存に使用


    if next_model is False:
        
        dataset_train2 = SegmentationDataSet0(
            inputs=input_dataset,        # train_preds
            targets=label_dataset,       # train_targets
            transform=None
        )

        dataset_val = SegmentationDataSet0(
            inputs=input_dataset_val,    # val_preds
            targets=label_dataset_val,   # val_targets
            transform=None
    )
    else:

        dataset_train2 = SegmentationDataSet0(
            inputs=input_dataset,        # original images
            targets=label_dataset,       # GT masks
            transform=transforms_training
        )

        dataset_val = SegmentationDataSet0(
            inputs=input_dataset_val,    # original validation images
            targets=label_dataset_val,   # GT masks
            transform=transforms_val
        )

    

    dataloader_training2 = DataLoader(dataset=dataset_train2,
                                        batch_size = 2,
                                        shuffle=True)
                                        #num_workers=4,
                                        #pin_memory=True)
        #もとはシャッフルtrue
    
        


    batch = next(iter(dataloader_training2))
  
    x, y = batch
    print("x.shape = ", x.shape)
    print("x.min(), x.max() = ", x.min(), x.max())
    print("y.shape = ", y.shape)
    print("torch.unique(y) = ", torch.unique(y))


    
    # # dataset training
    # dataset_val = SegmentationDataSet0(inputs=input_dataset_val,
    #                                     targets=label_dataset_val,
    #                                     transform=transforms_val)
    # if next_model==False:
    #     dataset_val = SegmentationDataSet0(inputs=input_dataset_val,
    #                                     targets=label_dataset_val,
    #                                     transform=None)
        
    #書き換え箇所
    dataloader_val = DataLoader(dataset=dataset_val,
                                     batch_size = 2,
                                     shuffle=False)
                                     #num_workers=4, # for faster training loads next batch while training
                                     #pin_memory=True)  # for faster training, keeps data in pinned memory
    
    
    
    ###earlystopping あり

    from unet import UNet
    from trainer2 import Trainer2 
    from torch import nn #import torch 
    from pytorchtools import EarlyStopping
    from torch.nn import BCEWithLogitsLoss
    from customLoss import DiceCELoss
    from customLoss import DiceLoss
    from customLoss import ExponentialLogCE_DiceLoss
    from customLoss import EnsembleInspiredLoss
    from customLoss import ConfusionPenaltyLoss
    from customLoss import CombinedLoss

    #device
    if torch.cuda.is_available():
        device = torch.device('cuda') 
    else: 
        torch.device('cpu')
        print("Before creating the UNet model: GPU was not available and CPU will be used instead")

    # custom logging the parameters of the UNet
    inChannels = inChannels
    outChannels = 11
    nBlocks = N_BLOCK
    startFilters = 32

    from customLog import custom_logger
    
    #custom_logger("/log/customLog.log", inChannels, outChannels, nBlocks, startFilters)

    #model
    model = UNet(in_channels = inChannels,
                 out_channels = outChannels,
                 n_blocks = nBlocks, 
                 start_filters=32,
                 activation='relu',
                 normalization='instance',
                 conv_mode='same',
                 dim=2,
                 ).to(device)
    
    # printing parameter name of the model could be done for criterion too
    # for name, param in model.named_parameters():
    #     print(name, param.device, param.requires_grad)
        
        


    # Assuming input_tensor is a sample input tensor with the correct shape (e.g., torch.randn(1, 3, 100, 100))
    input_tensor = torch.randn(1, 45, 100, 100).to(device)  # Adjust the shape as needed
    # print_model_shapes(model, input_tensor)

    from customLoss import BoundaryDiceLoss

    from customLoss import BoundaryIOU
    from customLoss import BoundaryDiceLoss
    from customLoss import FocalLoss
    from customLoss import CombinedLossV2
    from customLoss import CombinedLoss
    from customLoss import DiceLoss
    from customLoss import EnsembleInspiredLoss
    from customLoss import HausdorffLoss
    from customLoss import JointCELossHausdorff

    
    criterion = torch.nn.CrossEntropyLoss()
    

    #optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    #trainer
    trainer = Trainer2(model=model, 
                       device=device, 
                       criterion=criterion, 
                       optimizer=optimizer, 
                       training_DataLoader=dataloader_training2,
                       #validation_DataLoader=None, 
                       validation_DataLoader=dataloader_val, 
                       lr_scheduler=None, 
                       epochs=200, ##😺😺😺😺 epoch=0, 
                       notebook=True,
                       mine_epoch = MINE_EPOCH,
                       max_mine_train = MAX_MINE_TRAIN,
                       max_mine_val = MAX_MINE_VAL,
                       min_error_threshold = MIN_ERROR_THRESHOLD
                       )
    print("=======start training======")
    
    # start training
    #for combined
    training_losses, validation_losses, lr_rates, learning_curve, train_preds, train_targets, val_preds, val_targets = trainer.run_trainer()
    trainer.plot_mined_samples(n=10, split='train')
    trainer.plot_mined_samples(n=10, split='val')
    

    def safe_cat(tensors):
        if isinstance(tensors, torch.Tensor):
            return tensors  # already a tensor
        if len(tensors) == 0:
            raise RuntimeError("No predictions collected.")
        return torch.cat(tensors, dim=0)

    train_preds = safe_cat(train_preds)  # [N, C, H, W]
    train_targets = safe_cat(train_targets)  # [N, H, W]

    val_preds = safe_cat(val_preds)
    val_targets = safe_cat(val_targets)



    
    record_results(training_losses, validation_losses, lr_rates, learning_curve, model, fold, input_dataset_val, input_name_val, type_number, device, model_n)
    
    # Force garbage collection
    gc.collect()

    # Clear CUDA memory
    torch.cuda.empty_cache()

    # Optionally reset max memory tracking
    torch.cuda.reset_peak_memory_stats()
    
    if next_model == False:
        return
    
    inChannels = 11
    
    return Learn_EA(train_preds, train_targets, val_preds, val_targets, type_number, fold, next_model = False, model_n="unet2", inChannels=11)
    
    # #logging to csv
    # assert len(training_losses) == len(validation_losses) == len(lr_rates), \
    #     "Lengths of training, validation losses, and learning rates must match."

    # # Create dataframe
    # df = pd.DataFrame({
    #     "Epoch": list(range(1, len(training_losses) + 1)),
    #     "Training_Loss": training_losses,
    #     "Validation_Loss": validation_losses,
    #     "Learning_Rate": lr_rates
    # })

    # # Save to CSV
    # df.to_csv(test_result_dir + f"/fold{fold}_log.csv", index = False)
    
    # learning_curve.savefig(test_result_dir + f"/fold{fold}_learningcurve.png")
    
    # print("***************************")

    
    
    
    # #ここがちゃんとESで最適なエポック数のモデルになっているか要検証
    # model_dir = os.path.join("model", date_str)
    # if not os.path.exists(model_dir):
    #     os.makedirs(model_dir)
    # model_name = f"model_fold_{fold}.pt"
    # model_path = os.path.join(model_dir, model_name)
    # torch.save(model.state_dict(), model_path)
    # print(f"modelname:{model_name}を保存しました")
    # #torch.cuda.empty_cache()
    

    # model_weights = torch.load(model_path)
    # model.load_state_dict(model_weights)
    
    

    # # images = annealing_input_dataset
    # images = input_dataset_val

    # from inference import predict
    # from transformations import normalize_01, re_normalize
    # # predict the segmentation maps 
    # output = [predict(img, model, preprocess, postprocess, device) for img in images]


    # for i in range(len(input_name_val)):
    #     if(type_number == 0):
    #         cv2.imwrite(os.path.join(result_dir, f'{input_name_val[i]}.png'), output[i])
    #     elif(type_number == 1):
    #         cv2.imwrite(os.path.join(test_result_dir, 'result_original', f'{input_name_val[i]}.png'), output[i])
    #     elif(type_number == 2):
    #         cv2.imwrite(os.path.join(test_result_dir, 'result_test', f'{input_name_val[i]}.png'), output[i])



In [138]:
## dice calculation
import statistics
# Dicecの計算
def cal_DiceMulitple(dir, input_name):
    # change the label count as your preference
    Dice = [0] * 11
    Count1 = [0] * 11 #予測結果の各ラベルの要素数
    Count2 = [0] * 11 #ラベル画像の
    Count3 = [0] * 11 #正解した画素数
    
    for index in range(len(input_name)):
        print('index = ', index)

        img1 = cv2.imread(dir + '/' + input_name[index] + ".png", cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(labeled_dir + '/' + input_name[index] + ".png", cv2.IMREAD_GRAYSCALE)
        #_, img2 = cv2.threshold(img2, 0, 255, cv2.THRESH_BINARY)
        print("予測画像:", dir  + '/' + input_name[index] + ".png")
        print("テストラベル:", labeled_dir + '/' + input_name[index] + ".png")
        unique_label1 = np.unique(img1)
        unique_label2 = np.unique(img2)
        # print(unique_label1)
        # print(unique_label2)
        # change the image array size to your need
        for n in range(256):
            for l in range(256):
                value1 = img1[n,l]
                # for index, uq_value in enumerate(unique_label1):
                #     if(value1 == uq_value):
                #         value1 = index

                Count1[value1] += 1

                value2 = img2[n,l]
                Count2[value2] += 1                    

                if(value1 == value2):
                    Count3[value1] += 1 
    for i in range(11):
        if(Count1[i]+Count2[i] != 0):
            Dice[i] = (2*Count3[i])/(Count1[i] + Count2[i])
        if(Count1[i]+Count2[i] == 0):
            print("")
            #print("4 label case:" + str(input_name[index]))
    Dice.append(statistics.mean(Dice[1:]))
    print('Count1 = ', Count1)
    print('Count2 = ', Count2)
    print('Count3 = ', Count3)
    print('Dice = ', Dice)
    #print(unique_label)

    return Dice


In [139]:

## mConv_predict test function
import statistics
from inference import predict
from denseCRF import noiseReduction
# device
def mConv_predict(test_input_name, fold=0):
    print("*************************************Test*************************************")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ---------- Load Models ----------
    model_dir = os.path.join("model", date_str)
    model1_path = os.path.join(model_dir, f"unet1_fold_{fold}.pt")
    model2_path = os.path.join(model_dir, f"unet2_fold_{fold}.pt")

    model1 = UNet(in_channels=IN_CHANNEL,  # check the parameters
            out_channels=11,
            n_blocks=N_BLOCK,
            start_filters=32,
            activation='relu',
            normalization='instance', #use instance when "batch" size is less than 10? batch
            conv_mode='same',
            dim=2).to(device)
    
    model2 = UNet(in_channels=11,  # check the parameters
            out_channels=11,
            n_blocks=N_BLOCK,
            start_filters=32,
            activation='relu',
            normalization='instance', #use instance when "batch" size is less than 10? batch
            conv_mode='same',
            dim=2).to(device)


    model1.load_state_dict(torch.load(model1_path))
    model2.load_state_dict(torch.load(model2_path))

    model1.eval()
    model2.eval()

    # ---------- Load Test ----------
    test_input_dataset, _ = CreateWeightImage(test_input_name)

    output_stage1 = []
    output_stage2 = []

    # ---------- Stage-1 + Stage-2 ----------
    with torch.no_grad():
        for img in test_input_dataset:
            result1, prob1, pred1 = predict(img, model1, preprocess, postprocess, device)

            # Save stage-1 output (mask form)
            output_stage1.append(pred1)

            # prob1 shape: [1,11,H,W] → correct Stage-2 input
            prob1 = prob1.to(device)  
            out2 = model2(prob1)
            pred2 = torch.argmax(out2, 1).squeeze().cpu().numpy()
            output_stage2.append(pred2)

    # ---------- Save results ----------
    fold_dir = os.path.join(test_result_dir, f"fold{fold}")
    os.makedirs(fold_dir, exist_ok=True)

    unet1_dir = os.path.join(fold_dir, "unet1")
    unet2_dir = os.path.join(fold_dir, "unet2")
    os.makedirs(unet1_dir, exist_ok=True)
    os.makedirs(unet2_dir, exist_ok=True)

    for i, name in enumerate(test_input_name):
        cv2.imwrite(os.path.join(unet1_dir, f"{name}.png"), output_stage1[i])
        cv2.imwrite(os.path.join(unet2_dir, f"{name}.png"), output_stage2[i])

    # ---------- Dice + CRF ----------
    Dice1 = cal_DiceMulitple(unet1_dir, test_input_name)
    Dice2 = cal_DiceMulitple(unet2_dir, test_input_name)
    pd.DataFrame(Dice1).T.to_csv(f"{test_result_dir}/unet1Dice.csv", mode='a', header=False)
    pd.DataFrame(Dice2).T.to_csv(f"{test_result_dir}/unet2Dice.csv", mode='a', header=False)

    postCRF1 = noiseReduction(output_stage1, test_input_name, labeled_dir, unet1_dir, 0.8)
    postCRF2 = noiseReduction(output_stage2, test_input_name, labeled_dir, unet2_dir, 0.8)

    DiceCRF1 = cal_DiceMulitple(unet1_dir + "/crf", test_input_name)
    DiceCRF2 = cal_DiceMulitple(unet2_dir + "/crf", test_input_name)
    pd.DataFrame(DiceCRF1).T.to_csv(f"{test_result_dir}/unet1CRFDice.csv", mode='a', header=False)
    pd.DataFrame(DiceCRF2).T.to_csv(f"{test_result_dir}/unet2CRFDice.csv", mode='a', header=False)
    
    df1 = pd.DataFrame([Dice1])
    df2 = pd.DataFrame([Dice2])

    # Get last column value
    last_val1 = df1.iloc[0, -1]
    last_val2 = df2.iloc[0, -1]

    return last_val1, last_val2


In [140]:
import optuna
import os
from optuna.visualization import plot_optimization_history, plot_param_importances
import json
import traceback

STUDY_NAME = "unet_stage2"
DB_PATH = "sqlite:///unet_optuna.db"
BEST_JSON_PATH = "best_trial.json"
LOG_PATH = "training_log.txt"

def log(msg: str):
    print(msg)
    with open(LOG_PATH, "a") as f:
        f.write(msg + "\n")



In [141]:
from dataArrange import dataRearrange1
## Test Learn_EA without annealing
# training transformations and augmentations

transforms_training = ComposeDouble([
    FunctionWrapperDouble(create_dense_target, input=False, target=True),
    FunctionWrapperDouble(np.moveaxis, input=True, target=False, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

#追加箇所ver3
transforms_val =  ComposeDouble([
    FunctionWrapperDouble(create_dense_target, input=False, target=True),
    FunctionWrapperDouble(np.moveaxis, input=True, target=False, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)

])
def execute():
    ## Execution



    # random seed
    random_seed = 0

    X = file_names_with_prefix(data_dir, 'N')

    y_file_names = file_names_with_prefix(labeled_dir, 'N')


    # removal = ["N1-2", "N1-4", "N5-6", "N2-9"]
    # X = [item for item in X if item not in removal]
    # y_file_names = [item for item in y_file_names if item not in removal]

    y = [label_group[:2] for label_group in y_file_names]



    X = np.array(X)
    y = np.array(y)


    print(X)
    print(y)



    if CROSS_VAL:
        skf = StratifiedKFold(n_splits=N_SPLIT, shuffle=True)
        # each case same division
        #skf = StratifiedKFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
        for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
            # if fold == 1:
            #      continue
            # Split the data into train and test sets
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            # Split the training data further into train and validation ( 1/3 split)
            X_train_final, X_val, y_train_final, y_val = train_test_split(
                X_train, y_train, test_size=0.333333333, random_state=42, stratify=y_train)
            
            
            input_train = X_train_final
            input_name_val = X_val
            print("Cross validation: " + str(CROSS_VAL))
            print(f"Fold: {fold} out of {N_SPLIT}")
            print("Augmentation: " + str(AUGMENTED))
            if AUGMENTED:
                print("Augmentation amount: " + str(AUGMENTATION))
            print("Training: Total of " + str(len(input_train)) + " cases.")
            print(input_train)

            print("Validation: Total of " + str(len(input_name_val)) + " cases.")
            print(input_name_val)
            
            print("Test: Total of " + str(len(X_test)) + " cases.")  
            print(X_test)
            print()
            
            if AUGMENTED:
                ## This part is for accounting the data augmentation, attaches augmentation numbers after the original img
                ## ex: original number N1-1 -> N1-1-1, N1-1-2 etc.
                repeated_items_train = np.repeat(input_train, AUGMENTATION)
                suffixes_train = np.tile(np.arange(1, AUGMENTATION + 1), len(repeated_items_train))
                input_train = np.array([f"{item}-{suffix}" for item, suffix in zip(repeated_items_train, suffixes_train)])
                #Custom Arranging for training dataset because of the augmentation order
                input_train = np.array(dataRearrange1(input_train, AUGMENTATION))
                
                
                repeated_items_val = np.repeat(input_name_val, AUGMENTATION)
                suffixes_val = np.tile(np.arange(1, AUGMENTATION + 1), len(repeated_items_val))
                input_name_val = np.array([f"{item}-{suffix}" for item, suffix in zip(repeated_items_val, suffixes_val)])
            
            
            #input_dataset,label_dataset = CreateWeightImageNew(input_train, augmentation=AUGMENTED)
            input_dataset,label_dataset = CreateWeightImage(input_train, augmentation=AUGMENTED)
            input_dataset_val,label_dataset_val = CreateWeightImage(input_name_val, augmentation=AUGMENTED)
            
            
            Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val, 0, fold, next_model=True)
            mConv_predict(X_test, fold)
            
            # Force garbage collection 
            gc.collect()

            # Clear CUDA memory
            torch.cuda.empty_cache()

            # Optionally reset max memory tracking
            torch.cuda.reset_peak_memory_stats()
            
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=RANDOM_SEED)
        
        X_train_final, X_val, y_train_final, y_val = train_test_split(
                X_train, y_train, test_size=0.333333333, random_state=42, stratify=y_train)
        
        input_train = X_train_final
        input_name_val = X_val
        
        print("Cross validation: " + str(CROSS_VAL))
        print("Augmentation: " + str(AUGMENTED))
        if AUGMENTED:
            print("Augmentation amount: " + str(AUGMENTATION))
        print("Training: Total of " + str(len(input_train)) + " cases.")
        print(input_train)

        print("Validation: Total of " + str(len(input_name_val)) + " cases.")
        print(input_name_val)
            
        print("Test: Total of " + str(len(X_test)) + " cases.")  
        print(X_test)
        print()
        
        if AUGMENTED:
            ## This part is for accounting the data augmentation, attaches augmentation numbers after the original img
            ## ex: original number N1-1 -> N1-1-1, N1-1-2 etc.
            repeated_items_train = np.repeat(input_train, AUGMENTATION)
            suffixes_train = np.tile(np.arange(1, AUGMENTATION + 1), len(repeated_items_train))
            input_train = np.array([f"{item}-{suffix}" for item, suffix in zip(repeated_items_train, suffixes_train)])
            
            #Custom Arranging for training dataset because of the augmentation order
            input_train = np.array(dataRearrange1(input_train, AUGMENTATION))
                
                
            repeated_items_val = np.repeat(input_name_val, AUGMENTATION)
            suffixes_val = np.tile(np.arange(1, AUGMENTATION + 1), len(repeated_items_val))
            input_name_val = np.array([f"{item}-{suffix}" for item, suffix in zip(repeated_items_val, suffixes_val)])
        
        input_dataset,label_dataset = CreateWeightImage(input_train, augmentation=AUGMENTED)
        input_dataset_val,label_dataset_val = CreateWeightImage(input_name_val, augmentation=AUGMENTED)
        
        Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val, 0, next_model=True)
        dice1, dice2 = mConv_predict(X_test)
        

        
        # Force garbage collection
        gc.collect()

        # Clear CUDA memory
        torch.cuda.empty_cache()

        # Optionally reset max memory tracking
        torch.cuda.reset_peak_memory_stats()
        
    return dice1, dice2  

In [142]:
def objective(trial):
    try:
        # Hyperparameters for Optuna to explore
        
        MINE_EPOCH = trial.suggest_int("mine_epoch", 10, 40)
        MIN_ERROR_THRESHOLD = trial.suggest_int("min_error_threshold", 200, 1000)

        # -----------------
        # TRAINING CALL
        # -----------------
        dice1, dice2 = execute()
        trial.set_user_attr("dice1", dice1)

        log(f"Trial {trial.number} | "
            f"DICE1={dice1:.4f} | "
            f"DICE2={dice2:.4f} | "
            f"params={trial.params}")

        return dice2

    except Exception:
        err = traceback.format_exc()
        log(f"Trial {trial.number} FAILED\n{err}")
        # Fail gracefully and continue study
        return 0.0


In [143]:

import sys, os
import contextlib
from tqdm import tqdm
tqdm.disable = True
with open("train.log", "a") as log_file:
    with contextlib.redirect_stdout(log_file), \
         contextlib.redirect_stderr(log_file):

        study = optuna.create_study(
            study_name=STUDY_NAME,
            direction="maximize",
            storage=DB_PATH,
            load_if_exists=True,
        )

        try:
            study.optimize(objective, n_trials=200)
        except KeyboardInterrupt:
            log("=== INTERRUPTED BY USER ===")
        finally:
            log("=== OPTUNA FINISHED ===")


[I 2025-12-04 22:20:02,509] Using an existing study with name 'unet_stage2' instead of creating a new one.


Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Training:   0%|          | 0/11 [00:00<?, ?it/s]

Validation:   0%|          | 0/6 [00:00<?, ?it/s]

Training:   0%|          | 0/11 [00:00<?, ?it/s]

...

Training:   0%|          | 0/11 [00:00<?, ?it/s]

Validation:   0%|          | 0/6 [00:00<?, ?it/s]

Training:   0%|          | 0/11 [00:00<?, ?it/s]

Validation:   0%|          | 0/6 [00:00<?, ?it/s]

[W 2025-12-04 22:20:14,252] Trial 1 failed with parameters: {'mine_epoch': 17, 'min_error_threshold': 860} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/eric/anaconda3/envs/myenv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_3631502/4076041273.py", line 11, in objective
    dice1, dice2 = execute()
                   ^^^^^^^^^
  File "/tmp/ipykernel_3631502/2716774100.py", line 154, in execute
    Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val, 0, next_model=True)
  File "/tmp/ipykernel_3631502/3968670307.py", line 175, in Learn_EA
    training_losses, validation_losses, lr_rates, learning_curve, train_preds, train_targets, val_preds, val_targets = trainer.run_trainer()
                                                                                                                 

In [ ]:
# Save best trial information
best = study.best_trial
log(f"Best Trial = {best.number}")
log(f"Best DICE2 = {best.value:.4f}")
log(f"Best DICE1 = {best.user_attrs['dice1']:.4f}")
log(f"Best Params = {best.params}")

with open("best_trial.json", "w") as f:
    json.dump({
        "best_trial": best.number,
        "dice2": best.value,
        "dice1": best.user_attrs['dice1'],
        "params": best.params
    }, f, indent=4)


Best Trial = 0
Best DICE2 = 0.5766
Best DICE1 = 0.6578
Best Params = {'mine_epoch': 10, 'min_error_threshold': 857}


In [ ]:
df = study.trials_dataframe(attrs=("params", "user_attrs", "value"))
df.to_csv("optuna_results.csv", index=False)


In [ ]:
fig1 = optuna.visualization.plot_optimization_history(study)
fig1.show()

fig2 = optuna.visualization.plot_param_importances(study)
fig2.show()

fig3 = optuna.visualization.plot_parallel_coordinate(study)
fig3.show()

fig4 = optuna.visualization.plot_contour(study)
fig4.show()

ValueError: Cannot evaluate parameter importances with only a single trial.